# __slots__魔法

**`__slots__`的主要作用：**
- 通过在类定义中指定一个元组，明确列出允许的属性名称，可以防止动态添加其他属性。
- Python 类的实例属性存储在`__dict__`字典中，会占用较多内存；使用`__slots__`后，Python 不会为实例创建`__dict__`，而是将属性存储在固定大小的数组中，从而显著减少内存占用。
- 使用`__slots__`时，由于取消了`__dict__`，每个实例节省了字典的内存开销，属性存储也更加紧凑，属性访问更快。
- `__slots__`是一个非常强大的工具，尤其适用于需要处理大量实例的场景，合理使用它可以显著提升程序的性能和内存效率。

**注意事项：**
- `__slots__`仅对当前类实例起作用，对继承的子类无效。如果需要在子类中限制属性，子类也需要定义自己的`__slots__`。

**示例与性能对比：**

In [1]:
import sys
import time
import tracemalloc

class OrderWithDict:
    def __init__(self, order_id, symbol, price, quantity):
        self.order_id = order_id
        self.symbol = symbol
        self.price = price
        self.quantity = quantity

class OrderWithSlots:
    __slots__ = ('order_id', 'symbol', 'price', 'quantity')

    def __init__(self, order_id, symbol, price, quantity):
        self.order_id = order_id
        self.symbol = symbol
        self.price = price
        self.quantity = quantity

# 测试内存和速度
def measure_memory_and_speed(cls, n_objects=1_000_000):
    tracemalloc.start()
    start_time = time.time()
    objects = [cls(i, "BTC", 30000, 1) for i in range(n_objects)]
    creation_time = time.time() - start_time
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return {
        "内存占用(MB)": peak / 1024 / 1024,
        "对象创建时间(秒)": creation_time,
    }

print("测试普通类:", measure_memory_and_speed(OrderWithDict))
print("测试使用__slots__的类:", measure_memory_and_speed(OrderWithSlots))

测试普通类: {'内存占用(MB)': 179.7538480758667, '对象创建时间(秒)': 2.551945686340332}
测试使用__slots__的类: {'内存占用(MB)': 95.85917472839355, '对象创建时间(秒)': 1.690474033355713}


# 类方法与静态方法

- 类方法的第一个参数为cls，可以通过cls.访问类的属性和类的方法。
- 类方法可以通过类名调用，也可以通过实例调用。
- 在类方法中，可以通过cls()（cls代表当前类）创建类的实例（调用类的构造函数（__init__ 方法））。
- 在继承时，可以通过子类名调用父类的类方法，cls参数会自动指向子类。

<br/>

- 静态方法不接收任何隐式的第一个参数，既不接收cls，也不接收self，完全独立于类和实例，不能访问类属性或实例属性、类方法或实例方法，只能访问传递给它的参数。
- 静态方法可以通过类名调用，也可以通过实例调用。
- 在继承时，静态方法不会自动绑定到子类或父类，它的行为完全独立于类的继承结构；可以通过子类名调用父类的静态方法。
- 静态方法常用于工具函数或辅助函数。

In [2]:
from time import time, localtime

class Student:
    number = 0  # 类属性
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @classmethod   
    def counter(cls):
        cls.number += 1
        print(cls.number)

    @classmethod
    def show1(cls):
        return cls("wang", 27)

    @classmethod
    def show2(cls):
        print("Parent_class", cls.__name__)

    @staticmethod
    def nowtime():
        ctime = localtime(time())
        print(f"现在的时间：{ctime.tm_hour}时{ctime.tm_min}分{ctime.tm_sec}秒")

    @staticmethod
    def add(a, b):
        print(a + b)


class Smallstudent(Student):
    pass

In [3]:
obj1 = Student("Liu", 25)
print(obj1.name)
obj1.counter()  # 通过实例调用类方法

obj2 = Student("Zhang", 26)
print(obj2.name)
Student.counter()  # 通过类名调用类方法

obj3 = Student.show1()  # 通过cls()创建实例
print(obj3.name)
Student.counter()

Student.show2()
Smallstudent.show2()  # 通过子类名调用父类方法，cls参数自动指向子类

print("\n")

obj1.nowtime()  # 通过实例调用静态方法
Student.nowtime()  # 通过类名调用静态方法
Student.add(3, 4)  # 访问传递的参数，执行静态方法
Smallstudent.nowtime()  # 通过子类名调用静态方法

Liu
1
Zhang
2
wang
3
Parent_class Student
Parent_class Smallstudent


现在的时间：14时46分51秒
现在的时间：14时46分51秒
7
现在的时间：14时46分51秒


# 类的继承和多态

子类在继承了父类的方法后，可以对父类已有的方法给出新的实现版本，这个动作称之为方法重写（override）。通过方法重写我们可以让父类的同一个行为在子类中拥有不同的实现版本，当我们调用这个经过子类重写的方法时，不同的子类对象会表现出不同的行为，这个就是多态（poly-morphism）。

**在下面的例子中：** 
- 我们将Pet类处理成了一个抽象类，所谓抽象类就是不能够创建对象的类，这种类的存在就是专门为了让其他类去继承它。
- Python从语法层面并没有像Java或C#那样提供对抽象类的支持，但是我们可以通过abc模块的ABCMeta元类和abstractmethod包装器来达到抽象类的效果，如果一个类中存在抽象方法那么这个类就不能够实例化（创建对象）。
- 上面的代码中，Dog和Cat两个子类分别对Pet类中的make_voice抽象方法进行了重写并给出了不同的实现版本，实例化并调用该方法时，这个方法就表现出了多态行为（同样的方法做了不同的事情）。

**下面的例子中，子类没有显式定义`__init__`方法：**
- 这是因为Pet父类的`__init__`方法已经足够通用，可以直接用于初始化Dog和Cat子类的实例。
- 在类继承中，如果子类没有显式定义`__init__`方法，Python 会自动调用父类的`__init__`方法来初始化对象。

In [4]:
from abc import ABCMeta, abstractmethod

class Pet(object, metaclass=ABCMeta):
    """宠物"""
    def __init__(self, nickname):
        self._nickname = nickname

    @abstractmethod
    def make_voice(self):
        """发出声音"""
        pass


class Dog(Pet):
    """狗"""
    def make_voice(self):
        print(f"{self._nickname}：汪汪汪...")


class Cat(Pet):
    """猫"""
    def make_voice(self):
        print(f"{self._nickname}：喵...喵...")


pets = [Dog('旺财'), Cat('凯蒂')]
for pet in pets:
    pet.make_voice()

旺财：汪汪汪...
凯蒂：喵...喵...


# 类的封装

指的是将对象的属性（数据）和行为（方法）组合成一个独立的单元（类），并通过访问控制来隐藏内部实现细节，只暴露必要的接口供外部使用。封装的目的是提高代码的可维护性、可重用性和安全性。

**在下面的例子中：**
- self.__owner和self.__balance是私有属性，外部代码不应该直接访问它们。
- 通过@property提供只读访问接口，确保外部代码无法直接修改这些属性。
- 方法封装：
    - deposit方法封装了存款的逻辑，隐藏了内部实现细节。
    - @setter封装了owner属性赋值逻辑，隐藏内部实现细节。
- 通过封装，隐藏了内部实现细节，防止外部代码直接操作内部数据，增强代码的安全性和可维护性。例如，外部代码无法直接修改balance，只能通过deposit方法进行操作。

**@property和@setter**
- 如果只定义 @property，不定义 @setter，那么属性就是只读的。
- 如果同时定义 @property 和 @setter，那么属性就是可读写的，并且可以通过 @setter 方法对属性值进行验证和控制。

**单下划线和双下划线：**
- 单下划线 _ 是一种约定，表示该属性或方法是“受保护的”（protected），但它并不是真正的私有属性或方法，在类的外部仍然可以访问这些属性或方法，但按照约定，外部代码不应该直接访问它们。通常用于表示内部实现细节，提醒开发者这些属性或方法不应该被外部直接使用。
- 双下划线 __ 是真正的私有属性或方法，在类的外部无法直接访问这些属性或方法，从而实现真正的私有化。

In [5]:
class BankAccount:
    def __init__(self, name, amount=0):
        """私有属性，类外部不能直接访问"""
        self.__owner = name
        self.__balance = amount

    @property
    def owner(self):
        """使用装饰器@property将方法伪装成属性，在类外部以只读方式访问私有属性，但不允许外部修改属性值"""
        return self.__owner

    @owner.setter
    def owner(self, name):
        """使用@owner.setter封装属性赋值逻辑，实现在类外部修改self.__owner属性值"""
        if not isinstance(name, str) or len(name) == 0:
            raise ValueError("name must be a non-empty string")
        self.__owner = name
    
    @property
    def balance(self):
        """使用装饰器@property将方法伪装成属性，在类外部以只读方式访问私有属性，但不允许外部修改属性值"""
        return self.__balance

    def deposit(self, amount):
        """使用deposit()方法封装存款的逻辑，实现在类外部修改self.__balance属性值"""
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self.__balance += amount


# 创建初始实例
account = BankAccount("Alice", 1000)  
print(f"Account owner: {account.owner}") 
print(f"Account balance: {account.balance}")

print("\n")

# 更改owner的name和balance的amount
account.owner = "Liu"  # 在类外部修改self.__owner属性值
print(f"Account owner: {account.owner}") 
account.deposit(500)  # 在类外部修改self.__balance属性值
print(f"Account balance: {account.balance}")

Account owner: Alice
Account balance: 1000


Account owner: Liu
Account balance: 1500


In [6]:
class BankAccount:
    def __init__(self, name, amount=0):
        self._owner = name
        self.__balance = amount

account = BankAccount("zhang", 2000)
account._owner  # 不是真正的私有属性，在类的外部仍可直接访问

'zhang'

In [7]:
account.__balance  # 真正的私有属性，在类的外部不可直接访问，会报错

AttributeError: 'BankAccount' object has no attribute '__balance'